In [1]:
import os
import shutil

# = = = = = = = = = = = = = = =

if os.path.exists('./file/11.kit_collate'):
    shutil.rmtree('./file/11.kit_collate')

os.makedirs('./file/11.kit_collate')

# 1. Json

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/10.kit_info'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/10.kit_info/{file}',
                       header=0,
                       dtype=str).fillna('')
    
    # = = = = = = = = = = = = = = =

    list_column, list_column_price, list_column_note, list_column_note_2, list_column_specification = input_.columns.tolist(), ['Kit_Price'], ['Kit_Note 1'], ['Kit_Note 2'], []

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
        df_temp, dict_price, dict_note, dict_note_2, dict_dict = pd.DataFrame([input_.iloc[i]]).reset_index(drop=True), json.loads(input_.loc[i, 'Kit_Json_Price']), json.loads(input_.loc[i, 'Kit_Json_Note 1']), json.loads(input_.loc[i, 'Kit_Json_Note 2']), json.loads(input_.loc[i, 'Kit_Json_Specification'])

        df_temp.loc[0, 'Kit_Price'] = dict_price['0']

        dict_price = dict_price.get('1', {})
        for key, value in dict_price.items():
            key = f'Kit_Price-{int(key) + 2}'

            if key not in list_column_price:
                list_column_price.append(key)
                
            df_temp.loc[0, key] = value
        
        # = = = = = = = = = = = = = = =

        if dict_note:
            for key, value in dict_note.items():
                if key == '0':
                    df_temp.loc[0, 'Kit_Note 1'] = dict_note['0']
                else:
                    key = f'Kit_Note 1-{int(key) + 1}'
    
                    if key not in list_column_note:
                        list_column_note.append(key)
                        
                    df_temp.loc[0, key] = value
        else:
            df_temp.loc[0, 'Kit_Note 1'] = ''

        # = = = = = = = = = = = = = = =

        if dict_note_2:
            for key, value in dict_note_2.items():
                if key == '0':
                    df_temp.loc[0, 'Kit_Note 2'] = dict_note_2['0']
                else:
                    key = f'Kit_Note 2-{int(key) + 1}'
    
                    if key not in list_column_note_2:
                        list_column_note_2.append(key)
                        
                    df_temp.loc[0, key] = value
        else:
            df_temp.loc[0, 'Kit_Note 2'] = ''

        # = = = = = = = = = = = = = = =
        
        for dict_ in dict_dict.values():
            dict_['0'] = f'''Kit_{dict_['0']}'''
            
            if dict_['0'] in list_column:
                dict_['0'] = f'''{dict_['0']} (Specification)'''

            if dict_['0'] not in list_column_specification:
                list_column_specification.append(dict_['0'])
                
            df_temp.loc[0, dict_['0']] = dict_.get('1', 'N/A')
            
    
        list_df.append(df_temp)
    
    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['Kit_JOIN_MPNTCPC'],
                                ascending=[True],
                                ignore_index=True)

    output[list_column[:8] + list_column_price + list_column_note + list_column_note_2 + sorted(list_column_specification) + list_column[8:]].to_excel(f'./file/11.kit_collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 41/41 [00:00<00:00, 251.51it/s]



Done ~


# 2. Compatible

In [3]:
import pandas as pd

from tqdm import tqdm

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/9.kit_vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/9.kit_vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['Vehicle No'] = input_['Vehicle No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)

    # = = = = = = = = = = = = = = =

    input_2 = input_.drop_duplicates(['Kit_JOIN_MPNTCPC'],
                                     keep='first',
                                     ignore_index=True)
    input_2 = input_2.sort_values(by=['Kit_Part Number'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp = input_[input_['Kit_JOIN_MPNTCPC'] == input_2.loc[i, 'Kit_JOIN_MPNTCPC']].reset_index(drop=True)

        dict_vehicle = {}
        for j in range(len(df_temp)):
            make, model, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Year']
    
            if make not in dict_vehicle:
                dict_vehicle[make] = {model: [year]}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            elif year not in dict_vehicle[make][model]:
                dict_vehicle[make][model].append(year)
    
        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make in dict_vehicle:
            for model in dict_vehicle[make]:
                year_begin = min(dict_vehicle[make][model])
                year_end = max(dict_vehicle[make][model])
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        df_temp = pd.DataFrame([{'Kit_JOIN_MPNTCPC': input_2.loc[i, 'Kit_JOIN_MPNTCPC'],
                                 'Kit_Part Number': input_2.loc[i, 'Kit_Part Number'],
                                 'Vehicle': '\n'.join(sorted(list_vehicle))}])
        list_df.append(df_temp)

    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['Kit_JOIN_MPNTCPC'],
                                ascending=[True],
                                ignore_index=True)

    output.to_excel(f'./file/11.kit_collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 41/41 [00:00<00:00, 531.06it/s]


Done ~
